In [8]:
import pandas as pd
import numpy as np

price = pd.read_csv('price_processed.csv')
production = pd.read_csv('production_values.csv')
climate = pd.read_csv('climate_processed.csv')

dataframes = [price, production, climate]

for df in dataframes:
    df.set_index('Year', inplace=True)

price['Average Price Index'] = price.mean(axis=1)

climate['Average Mean Temperature (C) [June - Nov]'] = climate.iloc[:, 0:2].mean(axis=1)
climate['Average Total Precipitation (mm) [June - Nov]'] = climate.iloc[:, 2:4].mean(axis=1)

average_price = price[['Average Price Index']]
average_climate_temp = climate[['Average Mean Temperature (C) [June - Nov]']]
average_climate_precip = climate[['Average Total Precipitation (mm) [June - Nov]']]

df = pd.concat([production, average_price, average_climate_temp, average_climate_precip], axis=1)
df = df.dropna()

df = df.rename(columns={df.columns[0]: 'Quantity', 
                        df.columns[1]: 'Price', 
                        df.columns[2]: 'Temp', 
                        df.columns[3]: 'Rainfall'})

df['Quantity'] = df['Quantity'].str.replace(',', '').astype(float)
df['Log_Price'] = df['Price'].apply(lambda x: np.log(x))
df['Log_Quantity'] = df['Quantity'].apply(lambda x: np.log(x))

In [19]:
from linearmodels.iv import IV2SLS

formula = 'Log_Price ~ [Log_Quantity ~ Temp + Rainfall]'
model = IV2SLS.from_formula(formula, df)
results = model.fit()

print(results.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:              Log_Price   R-squared:                     -1.3201
Estimator:                    IV-2SLS   Adj. R-squared:                -1.4361
No. Observations:                  22   F-statistic:                    0.6585
Date:                Tue, Dec 05 2023   P-value (F-stat)                0.4171
Time:                        03:07:29   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                              Parameter Estimates                               
              Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
--------------------------------------------------------------------------------
Intercept       -17.915     27.720    -0.6463     0.5181     -72.245      36.414
Log_Quantity     1.3768     1.6967     0.811